In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model, clone_model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_value, get_optimal_action, get_optimal_actions, get_model_actions, test_model, one_batch, test_model_accuracy, one_batch_supervised, train, train_supervised, create_model


# 单步奖励收集

这是一个用于练习策略梯度法 (policy gradient method) 的玩具题。  
8个格子首尾相连，其中随机3个格子奖励，奖励数值为0~1的均匀随机数。  
动作：选择一个格子。
计分规则：  
1. 如果格子有奖励（大于0），获得对应的奖励。
2. 如果格子是空的（等于0），且两侧的格子都有奖励，会获得两侧格子的总奖励。
3. 否则奖励为0。

## 试验场

In [3]:
model = create_model(n_hidden_layers=1, n_dense_units=512, ratio_dropout=0.5, optimizer='adam')

In [4]:
best_idx, best_score, best_weights = train(model, 200, verbose = 1)

0 0.4196964480362878
1 0.533370590878932
2 0.5609241545276226
3 0.5788089194138847
4 0.6162808826571229
5 0.6452382955228987
6 0.655994152861143
7 0.6722184017389673
8 0.6987733853780318
9 0.7171190232594449
10 0.7223539906471552
11 0.7033830109012923
12 0.6959905332599443
13 0.7044713307648616
14 0.7143880538038173
15 0.7154319676800828
16 0.7097366429164729
17 0.7067603394464773
18 0.7058744872277171
19 0.7443789022555416
20 0.7775803586761424
21 0.7762757632392603
22 0.7747033627363588
23 0.7781361266069076
24 0.783626005851184
25 0.788986649470874
26 0.7768568303450715
27 0.7676763491307047
28 0.7875435922075503
29 0.7685812257518093
30 0.7461966768516559
31 0.7541715711196213
32 0.7439933785816666
33 0.7635545243688332
34 0.7953988900566931
35 0.7802354797650374
36 0.7768875566808764
37 0.7761849346229717
38 0.8079974398336426
39 0.8120313718200692
40 0.8113655419591916
41 0.8078333821925159
42 0.8117695504163437
43 0.8177050332178093
44 0.808231075415247
45 0.7989502032726911
46 

In [5]:
best_idx, best_score

(np.int64(88), np.float64(0.892805906106112))

In [6]:
model.set_weights(best_weights)

In [7]:
test_model(model, 10000)

np.float64(0.8903345454723381)

In [8]:
test_model_accuracy(model, 10000)

np.float64(0.4959)

## 一些测试结论

先做了一些模拟测试（10万次）确定评价模型的基准。如果采用最优解，单局游戏奖励的平均值为0.933，标准差0.347。采用随机策略的平均奖励是0.297。

首先初步对比了一下有无 BatchNormalization 和 Dropout 的情况，差距颇大。无 BatchNormalization 和 Dropout 时表现很差。观察 weights，应该是遇到了梯度消失和梯度爆炸。

之后的测试都用了 BatchNormalization 和 Dropout。


### 超参数

几个结论：
1. 对比强化学习和监督学习，强化学习算法的实现方式是正确的。
2. adam 比 sgd 更快，且更稳定（没有体现在上面的数据里），为什么？sgd模型的参数绝对值会越来越大，adam似乎没这个问题
3. 对于两个隐藏层的模型，sgd 比 adam 表现更好
4. 两个隐藏层的模型比一个隐藏层的模型效果差。为什么？
5. 为什么增大模型规模能提升模型效果和加快学习速度？看到过一个理论，说模型中大部分参数是没用的，只有部分参数起决定性作用，因为这部分参数的初始值很好。大的模型有更多的参数，有更大机会命中较好的初始参数。

之后调整测试 policy gradient method 采用超参数：optimalizer, n_hidden_layers, n_dense_units, ratio_dropout = 'adam', 1, 512, 0.5


观察了两个强化学习和监督学习模型，各自的得分是0.89和0.9，差距很小，最优解的命中率却是0.49和0.77。考虑：
1. 用 baseline 方法重新训练策略模型试试。
2. 训练过程中逐步降温。


### 降温

理论上是错误做法，实际效果也不好。


### 奖励 normalization

考虑以下行为
* 修改初始化时格子中随机数的取值区间
* 在获得的奖励上减去常数
* 在获得的奖励上乘以常数

理论上它们不应该影响策略，实际中它们对模型表现有重大影响。

所以，是否存在某个理想的适于模型学习的 G_t 的分布？

尝试在一个batch内对奖励作 normalization，效果较好。


* 有没有比 normalization 更好的办法？
* 用一个batch内的奖励的平均值和标准差做 normalization 合适吗？如果 batch_size 很小，感觉似乎不合适

怎么结合 normalization 和 baseline 呢？先在奖励上减去 baseline 再做 normalization？


### win_or_lose

命中最优解奖励1，否则-1。

加上归一化后，命中最优解的几率达到0.66，不过平均每局得分不见优势。

## 最大问题

不收敛。模型表现一开始变好，然后越来越差。为什么？假如说我现在正处于不断变差的阶段，继续训练，表现为什么不会提升？奖励信号为什么失效了？

尝试：
* 用test_model返回值做normalization
* 对每个state进行多次action抽样



## baseline

将 one_batch 中 
```
y_target[action] = run(state, action)
```
改为
```
y_target[action] = run(state, action) - 0.933
```

模型没有完全失效，训练过程中仍有提升，但只达到了0.72 左右。我看不出为什么在奖励上减去一个固定的数字会影响结果。